In [37]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import random

In [26]:
# Function that traverse through given directory and for every directory returns 
# lists of image and the corresponding emotion

def load_data(data_path):

    data_dir_list = os.listdir(data_path)

    img_data_list = []
    img_label_list = []

    # Traverse through each folder
    for dataset in data_dir_list:
        if os.path.isdir(data_path + '/' +  dataset):
            img_list=os.listdir(data_path+'/'+ dataset)
            
            # Resize each image
            for img in img_list:
                input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )

                # Convert image to grayscale
                input_img_gray = cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)

                input_img_resize=cv2.resize(input_img_gray,(48,48))
                img_data_list.append(input_img_resize)
                img_label_list.append(dataset)
                
    img_data = np.array(img_data_list)
    img_labels = np.array(img_label_list)

    return img_data, img_labels

In [27]:
# Normalize the pixel values to the range [0, 1]

def normalize_data(data):

    return data.astype('float32') / 255.0

In [31]:
# Because faces should be symetrical, but they aren't perfectly symetric, one possible way to make more data is
# to add mirrored images of faces. In this way, we are doubling instances of classes. We can exclude some classes
# from being processed this way 

    
def add_mirror_images(images, labels, exclude = []):
    
    all_images_list = []
    all_labels_list = []
    n = len(images)
    
    for i in range(n):
        
        all_images_list.append(images[i])
        all_labels_list.append(labels[i])
        
        # Don't perform mirroring if label is set to excluded
        if labels[i] not in exclude:
            # Mirror the image horizontally
            mirrored_image = cv2.flip(images[i], 1)
            all_images_list.append(mirrored_image)
            all_labels_list.append(labels[i]) 
        
    all_images = np.array(all_images_list)
    all_labels = np.array(all_labels_list)
    
    return all_images, all_labels 

In [9]:
# Assuming 'image' is your 48x48 image with values between 0 and 1

def plot_image(image, label):
    plt.imshow(image, cmap='gray') 
    plt.axis('off')  
    plt.show()
    print(label)

In [35]:
# Function for making Bar plot which shows count of every emotion in alphabetical order

def plot_count(images, labels):
    
    labels_set = set(labels)
    label_counts = {}
    
    for label in labels_set:
        label_counts[label] = 0
    
    for label in labels_set:
        for i in range(len(images)):
            if (labels[i] == label):
                label_counts[label] += 1
    
    # Sort dictionary in order to make same bar plot every time
    sorted_label_keys = sorted(label_counts.keys())
    sorted_label_counts = {key: label_counts[key] for key in sorted_label_keys}
    
    # Create a bar plot
    plt.figure(figsize=(10, 6)) 
    plt.bar(sorted_label_counts.keys(), sorted_label_counts.values(), color='skyblue')
    plt.xlabel('Emotion')
    plt.ylabel('Count')
    plt.title('Emotion Count')
    plt.xticks(rotation=45) 
    
    # Display the counts above each bar
    for i, count in enumerate(sorted_label_counts.values()):
        plt.text(i, count, str(count), ha='center', va='bottom')

    plt.tight_layout()
    plt.show()

In [63]:
# Function which will delete random instances from images if they exceed the number upto

def _decrease_bigger_indexes(target_images_indexes, random_index):
    for i in range(len(target_images_indexes)):
        if target_images_indexes[i] > random_index:
            target_images_indexes[i] -= 1
            
    return target_images_indexes    
    
def random_delete_upto(images, labels, target_label, upto):
    
    target_images_indexes = []
    data_images = images
    data_labels = labels
    
    # Finding all target images
    for i in range(len(images)):
        if target_label == labels[i]:
            target_images_indexes.append(i)
            
         
    # Randomly deleting images and corresponsive labels
    n = len(target_images_indexes)
    while upto < n:
        random_index = random.choice(target_images_indexes)
        target_images_indexes.remove(random_index)
        data_images = np.delete(data_images, random_index, axis=0)
        data_labels = np.delete(data_labels, random_index, axis=0)
        target_images_indexes = _decrease_bigger_indexes(target_images_indexes, random_index)
        n -= 1

    return data_images, data_labels